In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


## We are going to train logistic regression model and tfidf vectorizer and save them as .pkl files

In [1]:
import pandas as pd 
import numpy as np

In [2]:
imdb = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')


In [3]:
import re
import string

def clean_review(text):
    if not isinstance(text, str):
        return ""

    # Remove HTML tags
    text = re.sub(r'<.*?>', ' ', text)

    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [4]:
imdb['review'] = imdb['review'].apply(clean_review)


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
import torch
print("PyTorch GPU:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

PyTorch GPU: True
GPU name: Tesla P100-PCIE-16GB


In [6]:
vectorizer=TfidfVectorizer(max_features=5000)

In [7]:
review_vector=vectorizer.fit_transform(imdb['review'])

In [8]:
re_vec=review_vector.toarray()

In [9]:
imdb['review_vector']=list(re_vec)

In [39]:
!pip install xgboost==1.7.6 --quiet


^C


In [10]:
from sklearn.model_selection import train_test_split as tt

In [11]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
imdb['sentiment']=le.fit_transform(imdb['sentiment'])

In [12]:
x=imdb['review_vector']
y=imdb['sentiment']

In [13]:
x_train,x_test,y_train,y_test=tt(x,y,test_size=0.2,random_state=42)

In [14]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

In [15]:
import numpy as np
x_train_matrix = np.vstack(x_train.values)
x_test_matrix = np.vstack(x_test.values)
y_train_matrix=np.vstack(y_train.values)
y_test_matrix=np.vstack(y_test.values)

In [17]:
model = LogisticRegression(
    C=1.0,                    # Regularization strength (balanced)
    penalty='l2',             # L2 regularization (Ridge)
    solver='liblinear',       # Good for small to medium datasets
    max_iter=1000             # Ensure convergence
)
model.fit(x_train_matrix, y_train_matrix.ravel())

# Predict and evaluate
y_pred = model.predict(x_test_matrix)
print("✅ Accuracy:", accuracy_score(y_test_matrix, y_pred))
print("\n🔍 Classification Report:\n", classification_report(y_test_matrix, y_pred))



✅ Accuracy: 0.8956

🔍 Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.88      0.89      4961
           1       0.89      0.91      0.90      5039

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



In [18]:

import joblib

# Save the trained model
joblib.dump(model, 'logistic_model.pkl')

# Save the TF-IDF vectorizer
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

In [20]:
import shutil

# move to the correct directory just to be sure
shutil.move("logistic_model.pkl", "/kaggle/working/logistic_model.pkl")
shutil.move("tfidf_vectorizer.pkl", "/kaggle/working/tfidf_vectorizer.pkl")


'/kaggle/working/tfidf_vectorizer.pkl'